In [21]:
import pandas as pd
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
db_path = '/Users/zphilipp/git/research/dealsdb/deals_db.db'
prepositions_and_conjunctions = [
    "about", "above", "across", "after", "against", "along", "among", "around", "at",
    "before", "behind", "below", "beneath", "beside", "between", "beyond", "by",
    "during", "for", "from", "in", "inside", "into", "near", "of", "off", "on",
    "out", "outside", "over", "through", "throughout", "to", "toward", "under",
    "until", "up", "with", "within", "without", "and", "but", "or", "for", "nor",
    "so", "yet", "although", "because", "as", "since", "unless", "while", "when",
    "where", "after", "before", "the", "a"
]
pattern = r'\b(?:' + '|'.join(prepositions_and_conjunctions) + r')\b'

def remove_prepositions_and_conjunctions(text):
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'\d+', '', cleaned_text)
    return re.sub(r'\s+', ' ', cleaned_text).strip()

In [30]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

sql_query = """
    SELECT
        d.customer_category_id,
        c.name,
        GROUP_CONCAT(d.title, ',') || ' ' || GROUP_CONCAT(d.title_general, ',') AS text
    FROM deals d join customer_taxonomy c
        ON (d.customer_category_id=c.id)
    GROUP BY customer_category_id
"""

# Execute the query and load the data into a DataFrame
df = pd.read_sql_query(sql_query, conn)
conn.close()

In [31]:
def get_important_words(document):
    document = remove_prepositions_and_conjunctions(document)
    document = [document]

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(document)

    feature_names = vectorizer.get_feature_names_out()
    first_document_vector = tfidf_matrix[0]
    important_words = sorted(
        [(feature_names[i], first_document_vector[0, i]) for i in first_document_vector.nonzero()[1]],
        key=lambda x: x[1], reverse=True
    )
    return important_words

In [32]:
df['important_words'] = df['text'].apply(get_important_words)

In [36]:
df[['name', 'important_words']].head(200)

,name,important_words
0,Specialty Pillows,"[(oz, 0.3897498098073312), (women, 0.3375275186542559), (spray, 0.24451745670739009), (men, 0.23649934791886718), (silver, 0.2229731470060546), (de, 0.21356058451517987), (eau, 0.21146890396165216), (gold, 0.20867999655694855), (pack, 0.1913887706477861), (sterling, 0.16865917529945157), (white, 0.16412720076680817), (black, 0.14802126050464476), (earrings, 0.1379114711625941), (parfum, 0.12089913599390202), (set, 0.11560021192496514), (ml, 0.09698425499856847), (fl, 0.09656591888786292), (plated, 0.09126699481892604), (toilette, 0.09001198648680941), (stud, 0.08617723880534194), (ct, 0.08269110454946241), (round, 0.07913524760846528), (blue, 0.07683439899958479), (mm, 0.07648578557399684), (crystal, 0.06700350039800453), (edt, 0.06616682817659345), (xl, 0.06560904669565272), (gb, 0.06302930734630187), (edp, 0.060658736052303784), (box, 0.057869828647600166), (cut, 0.056823988370836305), (ring, 0.052292013838192916), (new, 0.052292013838192916), (bag, 0.051176450876311466), (piece, 0.04866643421207821), (apple, 0.0480389300460199), (dog, 0.04782976199066712), (size, 0.0472719805097264), (one, 0.04573808143713941), (sleeve, 0.04538946801155145), (socks, 0.043995014309199644), (diamond, 0.043228064772906145), (refurbished, 0.04267028329196542), (vibrator, 0.042112501811024695), (admission, 0.04183361107055433), (gift, 0.040508880053320116), (costway, 0.04036943468308493), (other, 0.03967220783190903), (rose, 0.03925387172120348), (crystals, 0.03848692218490999), (yellow, 0.038347476814674805), (pink, 0.037929140703969265), (solid, 0.03744108190814613), (all, 0.03730163653791095), (long, 0.03709246848255818), (red, 0.03695302311232299), (adjustable, 0.036813577742087815), (necklace, 0.03653468700161745), (waterproof, 0.03632551894626468), (pcs, 0.03625579626114709), (multi, 0.03611635089091191), (jewelry, 0.035767737465323954), (sizes, 0.035698014780206365), (cttw, 0.03548884672485359), (bracelet, 0.035349401354618414), (made, 0.03520995598438323), (high, 0.03514023329926564), (swarovski, 0.03444300644808974), (cubic, 0.03437328376297215), (zirconia, 0.034233838392736964), (grey, 0.03388522496714901), (color, 0.03374577959691383), (green, 0.033397166171325876), (floral, 0.03332744348620829), (scent, 0.0331879981159731), (pet, 0.03290910737550274), (pairs, 0.03269993932014997), (compression, 0.032630216635032384), (brass, 0.0324907712647972), (italian, 0.0324907712647972), (skin, 0.03137520830291575), (neck, 0.030677981451739847), (fresh, 0.030608258766622255), (paris, 0.030538536081504666), (sapphire, 0.030329368026151892), (unisex, 0.02886519163868249), (hair, 0.02865602358332972), (outdoor, 0.028307410157741766), (storage, 0.028237687472624177), (soft, 0.028167964787506584), (travel, 0.027958796732153814), (toy, 0.027958796732153814), (case, 0.027889074047036222), (spot, 0.02761018330656586), (dress, 0.027261569880977908), (heart, 0.02719184719586032), (band, 0.02719184719586032), (multiple, 0.027052401825625138), (sex, 0.026912956455389957), (wireless, 0.026773511085154775), ...]"
1,Eyebrow Embroidery,"[(eyebrow, 0.35583940116059926), (extensions, 0.3509648888159335), (set, 0.3070942777139418), (one, 0.29734525302461035), (full, 0.2583491542672844), (lash, 0.2486001295779529), (brows, 0.19985500613129548), (session, 0.19010598144196397), (beauty, 0.19010598144196397), (eyelash, 0.19010598144196397), (volume, 0.17548244440796676), (microblading, 0.16573341971863528), (your, 0.15598439502930378), (hybrid, 0.14136085799530657), (lashes, 0.12673732096130932), (classic, 0.12186280861664359), (look, 0.12186280861664359), (brow, 0.10723927158264634), (lamination, 0.09261573454864912), (natural, 0.09261573454864912), (permanent, 0.08774122220398338), (eyebrows, 0.07311768516998615), (touch, 0.07311768516998615), (mink, 0.0682431728253204), (experience, 0.0682431728253204), (transform, 0.0682431728253204), (makeup, 0.06336866048065466), (tint, 0.06336866048065466), (eyelashes, 0.05849414813598